In [3]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import csv



In [5]:
#Criando um dicionário com todos os caminhos dos arquivos. Isso vai ser bom porque utilizarei as chaves para criar uma coluna de "Ano"
arquivos = {
    '2019': 'data/dados_servidores_2019.csv',
    '2020': 'data/dados_servidores_2020.csv',
    '2021': 'data/dados_servidores_2021.csv'
}

lista_dataframes = [] #Para armazenar todos os dataframes

for ano, arquivo in arquivos.items():
    try:  
        df = pd.read_csv(arquivo, encoding="utf-8")

        lista_dataframes.append(df)

    except Exception as e:
        print(f"Erro ao ler os dados do ano {ano}: {e}")


df_raw = pd.concat(lista_dataframes, ignore_index=True) #Concatenando todos os DFs



print(f"Total de linhas e colunas no df unificado: {df_raw.shape}")

C:\Users\sfrei\AppData\Local\Temp\ipykernel_6920\3536511882.py:12: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo, encoding="utf-8")
C:\Users\sfrei\AppData\Local\Temp\ipykernel_6920\3536511882.py:12: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo, encoding="utf-8")
C:\Users\sfrei\AppData\Local\Temp\ipykernel_6920\3536511882.py:12: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(arquivo, encoding="utf-8")


Total de linhas e colunas no df unificado: (1398332, 34)


In [6]:
# Copia do df carregado para não precisarmos rodar a importação toda vez ue precisarmos
# voltar ao df original.
df = df_raw.copy()
df.head()

,asalseanoo,asalsemess,csalseccpf,csalsematr,nsalsenome,nsalseempr,nsalsecate,nsalsecarg,nsalsefunc,vsalsecarg,...,esalseunidade,dslderadmissao,dslserdesligamento,aslserjornadamensal,esalseinstrucao,esalsegenero,dsalseaposentadoria,esalseadministracao,eslserlotacao,eselsesituacao
0,2019,4,***040714**,4480,EVERALDO PADRE DA SILVA,CTTU,CELETISTAS,VIGILANTE,SEM INFORMACAO,1198.8,...,SETOR DE SERVICOS ADMINISTRATIVOS,1991-08-30,2021-11-01,220.0,Mdio Completo,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
1,2019,4,***105614**,4529,JOSE ARTUR SEABRA,CTTU,CELETISTAS,VIGILANTE,SEM INFORMACAO,1198.8,...,SETOR DE SERVICOS ADMINISTRATIVOS,1991-09-05,2021-10-01,220.0,Mdio Completo,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
2,2019,4,***507834**,512222,VICTOR FELIX SILVA MELO,CTTU,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,...,SETOR DE PROCESSAMENTO DE MULTAS,2019-01-16,2019-07-31,110.0,Mdio Incompleto,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
3,2019,4,***256834**,512230,DAIANNA M C MAIA DE OLIVEIRA,CTTU,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,...,GESTAO DE UNID. DE MOB. SUSTENTAVEL,2019-02-01,2019-11-30,110.0,Mdio Incompleto,Feminino,NaN,Indireta,CTTU,Desligado/Exonerado
4,2019,4,***026244**,512249,ENILTON RUAN M DOS SANTOS,CTTU,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,...,SETOR DE PROCESSAMENTO DE MULTAS,2019-02-01,2020-12-31,110.0,Mdio Incompleto,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado


In [7]:
dict_dados = [
      {
        "codigo": "asalseanoo",
        "descricao": "ANO",
      },
      {
        "codigo": "asalsemess",
        "descricao": "MES",

      },
      {
        "codigo": "csalseccpf",
        "descricao": "CPF",

      },
      {
        "codigo": "csalsematr",
        "descricao": "MATRICULA",

      },
      {
        "codigo": "nsalsenome",
        "descricao": "NOME",

      },
      {
        "codigo": "nsalseempr",
        "descricao": "SECRETARIA/ORGAO",

      },
      {
        "codigo": "nsalsecate",
        "descricao": "CATEGORIA",

      },
      {
        "codigo": "nsalsecarg",
        "descricao": "CARGO",

      },
      {
        "codigo": "nsalsefunc",
        "descricao": "FUNCAO",

      },
      {
        "codigo": "vsalsecarg",
        "descricao": "VENCIMENTO CARGO",

      },
      {
        "codigo": "vsalsefunc",
        "descricao": "VENCIMENTO FUNCAO",

      },
      {
        "codigo": "vsalseremu",
        "descricao": "REMUNERACAO",

      },
      {
        "codigo": "vsalseferi",
        "descricao": "VENCIMENTO FERIAS",

      },
      {
        "codigo": "vsalsenatl",
        "descricao": "VENCIMENTO NATAL",

      },
      {
        "codigo": "vsalseoutr",
        "descricao": "VENCIMENTOS OUTROS",

      },
      {
        "codigo": "vsalseprov",
        "descricao": "TOTAL PROVENTOS",

      },
      {
        "codigo": "vsalsedxcd",
        "descricao": "DESCONTOS EXCEDENTES",

      },
      {
        "codigo": "vsalsedrst",
        "descricao": "DESCONTO FALTA OU RESTITUICAO",

      },
      {
        "codigo": "vsalsedprv",
        "descricao": "DESCONTO PREVIDENCIA",

      },
      {
        "codigo": "vsalsedrrf",
        "descricao": "DESCONTO IRRF",

      },
      {
        "codigo": "vsalsedtot",
        "descricao": "DESCONTOS COMPULSORIOS",

      },
      {
        "codigo": "vsalseliqd",
        "descricao": "VENCIMENTO LIQUIDO",

      },
      {
        "codigo": "vsalsedife",
        "descricao": "DIFERENCA VENCIMENTO MES",

      },
      {
        "codigo": "tslserulat",
        "descricao": "DATA E HORA EVENTO",

      },
      {
        "codigo": "eslserlotacao",
        "descricao": "LOTACAO",

      },
      {
        "codigo": "dslderadmissao",
        "descricao": "DATA ADMISSAO",

      },
      {
        "codigo": "dslserdesligamento",
        "descricao": "DATA DESLIGAMENTO",

      },
      {
        "codigo": "aslserjornadamensal",
        "descricao": "JORANADA MENSAL",

      },
      {
          "codigo" : "esalseinstrucao" ,
          "descricao": "GRAU DE INSTRUCAO"
      },
      {
          "codigo" : "esalsegenero" ,
          "descricao": "GENERO"
      },
      {
          "codigo" : "eselsesituacao" ,
          "descricao": "SITUACAO"
      },
      {
          "codigo" : "esalseadministracao" ,
          "descricao": "TIPO ADM"
      }

      ]

### Padronizando as Colunas

#### Identificando Colunas

In [8]:
# Renomeando as colunas com base nas discrições do dicionario de dados para nomes mais intuitivos
for campo in dict_dados:
    df = df.rename(columns={campo['codigo']: campo['descricao']})

In [9]:
print('DataSet')
display(df.head(5))
print('Resumo de informações:')
display(df.info())
print('Verificando Valores únicos:')
display(df.nunique(dropna=False))
print('Verificando valores nulos:')
display(df.isnull().sum())

DataSet


,ANO,MES,CPF,MATRICULA,NOME,SECRETARIA/ORGAO,CATEGORIA,CARGO,FUNCAO,VENCIMENTO CARGO,...,esalseunidade,DATA ADMISSAO,DATA DESLIGAMENTO,JORANADA MENSAL,GRAU DE INSTRUCAO,GENERO,dsalseaposentadoria,TIPO ADM,LOTACAO,SITUACAO
0,2019,4,***040714**,4480,EVERALDO PADRE DA SILVA,CTTU,CELETISTAS,VIGILANTE,SEM INFORMACAO,1198.8,...,SETOR DE SERVICOS ADMINISTRATIVOS,1991-08-30,2021-11-01,220.0,Mdio Completo,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
1,2019,4,***105614**,4529,JOSE ARTUR SEABRA,CTTU,CELETISTAS,VIGILANTE,SEM INFORMACAO,1198.8,...,SETOR DE SERVICOS ADMINISTRATIVOS,1991-09-05,2021-10-01,220.0,Mdio Completo,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
2,2019,4,***507834**,512222,VICTOR FELIX SILVA MELO,CTTU,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,...,SETOR DE PROCESSAMENTO DE MULTAS,2019-01-16,2019-07-31,110.0,Mdio Incompleto,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
3,2019,4,***256834**,512230,DAIANNA M C MAIA DE OLIVEIRA,CTTU,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,...,GESTAO DE UNID. DE MOB. SUSTENTAVEL,2019-02-01,2019-11-30,110.0,Mdio Incompleto,Feminino,NaN,Indireta,CTTU,Desligado/Exonerado
4,2019,4,***026244**,512249,ENILTON RUAN M DOS SANTOS,CTTU,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,...,SETOR DE PROCESSAMENTO DE MULTAS,2019-02-01,2020-12-31,110.0,Mdio Incompleto,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado


Resumo de informações:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1398332 entries, 0 to 1398331
Data columns (total 34 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   ANO                            1398332 non-null  int64  
 1   MES                            1398332 non-null  int64  
 2   CPF                            1398332 non-null  object 
 3   MATRICULA                      1398332 non-null  int64  
 4   NOME                           1398332 non-null  object 
 5   SECRETARIA/ORGAO               1398332 non-null  object 
 6   CATEGORIA                      1398332 non-null  object 
 7   CARGO                          1398332 non-null  object 
 8   FUNCAO                         1398332 non-null  object 
 9   VENCIMENTO CARGO               1398332 non-null  float64
 10  VENCIMENTO FUNCAO              1398332 non-null  float64
 11  REMUNERACAO                    1398332 non-null  floa

None

Verificando Valores únicos:


ANO                                   3
MES                                  12
CPF                               35744
MATRICULA                         47933
NOME                              49371
SECRETARIA/ORGAO                     10
CATEGORIA                            15
CARGO                               613
FUNCAO                              126
VENCIMENTO CARGO                   5726
VENCIMENTO FUNCAO                   331
REMUNERACAO                       25462
VENCIMENTO FERIAS                 14472
VENCIMENTO NATAL                  36912
VENCIMENTOS OUTROS                63159
TOTAL PROVENTOS                  134297
DESCONTOS EXCEDENTES                704
DESCONTO FALTA OU RESTITUICAO      5507
DESCONTO PREVIDENCIA              22247
DESCONTO IRRF                     39108
DESCONTOS COMPULSORIOS            52618
VENCIMENTO LIQUIDO               121022
DIFERENCA VENCIMENTO MES          20767
DATA E HORA EVENTO                    3
esalseunidade                      1564


Verificando valores nulos:


ANO                                    0
MES                                    0
CPF                                    0
MATRICULA                              0
NOME                                   0
SECRETARIA/ORGAO                       0
CATEGORIA                              0
CARGO                                  0
FUNCAO                                 0
VENCIMENTO CARGO                       0
VENCIMENTO FUNCAO                      0
REMUNERACAO                            0
VENCIMENTO FERIAS                      0
VENCIMENTO NATAL                       0
VENCIMENTOS OUTROS                     0
TOTAL PROVENTOS                        0
DESCONTOS EXCEDENTES                   0
DESCONTO FALTA OU RESTITUICAO          0
DESCONTO PREVIDENCIA                   0
DESCONTO IRRF                          0
DESCONTOS COMPULSORIOS                 0
VENCIMENTO LIQUIDO                     0
DIFERENCA VENCIMENTO MES               0
DATA E HORA EVENTO                     0
esalseunidade   

### Algumas colunas não foram identificadas no dicionario de dados disponivel, vamos tentar identifica-las e analisar quais fazem sentido para nossa analise e quais podemos descartar.

In [10]:
print('Coluna esalseunidade:')
display(df['esalseunidade'].unique())
print('Coluna dsalseaposentadoria:') 
display(df['dsalseaposentadoria'].unique())


Coluna esalseunidade:


array(['SETOR DE SERVICOS ADMINISTRATIVOS',
       'SETOR DE PROCESSAMENTO DE MULTAS',
       'GESTAO DE UNID. DE MOB. SUSTENTAVEL', ...,
       'SEC. DE MEIO AMBIENTE E SUSTENTABILIDADE',
       'UNIDADE DE ANALISE DAS LICIT DE OBRAS E',
       'FUND. DE CULTURA CIDADE DO RECIFE'], shape=(1564,), dtype=object)

Coluna dsalseaposentadoria:


array([nan, '2013-10-18', '2013-12-22', ..., '2021-10-02', '2021-08-05',
       '2021-06-29'], shape=(2741,), dtype=object)

### Vamos Analisar A coluna 'esalseunidade', verificamos uma possivel correlação com outras colunas e queremos entender se podemos otimizar a visão do nosso dataset

In [11]:
print(df['esalseunidade'].unique())
print("")
print(df['SECRETARIA/ORGAO'].unique())
print("")
print(df['LOTACAO'].unique())

['SETOR DE SERVICOS ADMINISTRATIVOS' 'SETOR DE PROCESSAMENTO DE MULTAS'
 'GESTAO DE UNID. DE MOB. SUSTENTAVEL' ...
 'SEC. DE MEIO AMBIENTE E SUSTENTABILIDADE'
 'UNIDADE DE ANALISE DAS LICIT DE OBRAS E'
 'FUND. DE CULTURA CIDADE DO RECIFE']

['CTTU' 'URB RECIFE' 'FUND. DE CULTURA CIDADE RECIFE'
 'CIA. SERV. URBANO DO RECIFE' 'EMPRESA MUNIC. DE INFORMATICA'
 'PREFEITURA DA CIDADE DO RECIFE' 'FUNDO RECIFIN'
 'EMPRESA MANUT. LIMPEZA URBANA' 'FUNDO RECIPREV'
 'RECIPREV-AUT.PREV.MUN.RECIFE']

['CTTU' 'URB' 'SEC DE PLANEJAMENTO URBANO' 'GABINETE DE IMPRENSA'
 'FUN. CULTURA' 'CSURB' 'SEC DE POLITICA URBANA E LICENCIAMENTO' 'EMPREL'
 'CMR' 'EMLURB' 'SEC DE EDUCACAO' 'SEC DE SAUDE' 'RECIPREV'
 'SEC DE MEIO AMBIENTE E SUSTENTABILIDADE' 'SEC DE SEGURANCA CIDADA'
 'GABINETE DO PREFEITO' 'FUN. RECIFIN' 'FUN. RECIPREV'
 'SEC PLANEJAMENTO GESTAO TRANSFOR DIGITAL' 'SEC DE TURISMO E LAZER'
 'SEC DE SANEAMENTO' 'GABINETE DA VICE PREFEITA' 'SEC DA MULHER'
 'SEC DESENV SOCIAL DIR HUMAN JUVENTU DROG'
 'ASSE


é possivel perceber que 'SECRETARIA/ORGAO' está contido em 'LOTACAO' e que 'esalseunidade' é tão especifico com 1564 valores unicos que fica dificil visualizar a utilidade prática em nossas análises.

Conclusão: Iremos dropar as colunas SECRETARIA/ORGAO e esalseunide.

In [12]:
df = df.drop(columns=['SECRETARIA/ORGAO', 'esalseunidade'], axis=1) #Removendo as colunas
df.head()

,ANO,MES,CPF,MATRICULA,NOME,CATEGORIA,CARGO,FUNCAO,VENCIMENTO CARGO,VENCIMENTO FUNCAO,...,DATA E HORA EVENTO,DATA ADMISSAO,DATA DESLIGAMENTO,JORANADA MENSAL,GRAU DE INSTRUCAO,GENERO,dsalseaposentadoria,TIPO ADM,LOTACAO,SITUACAO
0,2019,4,***040714**,4480,EVERALDO PADRE DA SILVA,CELETISTAS,VIGILANTE,SEM INFORMACAO,1198.8,0.0,...,2024-01-11 17:27:56,1991-08-30,2021-11-01,220.0,Mdio Completo,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
1,2019,4,***105614**,4529,JOSE ARTUR SEABRA,CELETISTAS,VIGILANTE,SEM INFORMACAO,1198.8,0.0,...,2024-01-11 17:27:56,1991-09-05,2021-10-01,220.0,Mdio Completo,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
2,2019,4,***507834**,512222,VICTOR FELIX SILVA MELO,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,0.0,...,2024-01-11 17:27:56,2019-01-16,2019-07-31,110.0,Mdio Incompleto,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado
3,2019,4,***256834**,512230,DAIANNA M C MAIA DE OLIVEIRA,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,0.0,...,2024-01-11 17:27:56,2019-02-01,2019-11-30,110.0,Mdio Incompleto,Feminino,NaN,Indireta,CTTU,Desligado/Exonerado
4,2019,4,***026244**,512249,ENILTON RUAN M DOS SANTOS,ESTAGIARIOS,ESTAGIARIO,SEM INFORMACAO,0.0,0.0,...,2024-01-11 17:27:56,2019-02-01,2020-12-31,110.0,Mdio Incompleto,Masculino,NaN,Indireta,CTTU,Desligado/Exonerado


sobre a coluna dsalseaposentadoria: Não conseguimos indentificar com clareza ao que se refere a data e decidimos dropar a coluna

In [13]:
df = df.drop(columns='dsalseaposentadoria')


## Tratando valores nulos

In [14]:
df.isnull().sum()

ANO                                    0
MES                                    0
CPF                                    0
MATRICULA                              0
NOME                                   0
CATEGORIA                              0
CARGO                                  0
FUNCAO                                 0
VENCIMENTO CARGO                       0
VENCIMENTO FUNCAO                      0
REMUNERACAO                            0
VENCIMENTO FERIAS                      0
VENCIMENTO NATAL                       0
VENCIMENTOS OUTROS                     0
TOTAL PROVENTOS                        0
DESCONTOS EXCEDENTES                   0
DESCONTO FALTA OU RESTITUICAO          0
DESCONTO PREVIDENCIA                   0
DESCONTO IRRF                          0
DESCONTOS COMPULSORIOS                 0
VENCIMENTO LIQUIDO                     0
DIFERENCA VENCIMENTO MES               0
DATA E HORA EVENTO                     0
DATA ADMISSAO                         36
DATA DESLIGAMENT

In [15]:
checkpoint_1 = df.copy()

verificando os valores que foram identificados como null 

- DATA ADMISSAO e JORANADA MENSAL: temos 36 linhas NaN
- GENERO: Apenas 2 linhas
- GRAU DE INSTRUCAO

In [16]:
print("Valores únicos da coluna 'JORANADA MENSAL': \n", df['JORANADA MENSAL'].unique())
print("\nValores únicos da coluna 'DATA ADMISSAO': \n", df['DATA ADMISSAO'].unique())
print("\nValores unicos da coluna 'GRAU DE INSTRUCAO': \n", df['GRAU DE INSTRUCAO'].unique())
print("\nValores unicos da coluna 'GENERO': \n", df['GENERO'].unique())

Valores únicos da coluna 'JORANADA MENSAL': 
 [220. 110. 120. 200. 180. 145.   0. 270. 150. 160.  80. 100.  nan 240.
 250. 125. 260. 190. 140. 210. 115. 235. 255. 265.  32. 165. 195. 170.
 130. 225.  18.  20. 108.  40.   8. 185. 155. 230. 245. 135. 175. 300.
 147. 144. 215. 205.  10.]

Valores únicos da coluna 'DATA ADMISSAO': 
 ['1991-08-30' '1991-09-05' '2019-01-16' ... '2021-12-10' '2021-12-20'
 '2012-12-12']

Valores unicos da coluna 'GRAU DE INSTRUCAO': 
 ['Mdio Completo' 'Mdio Incompleto' 'Superior Incompleto'
 'Fundamental 2 Incompleto' 'Ps-graduao Completo' 'Superior Completo'
 'Fundamental 2 Completo' 'Mestrado Completo' 'Analfabeto'
 'Fundamental 1 Completo' 'Fundamental 1 Incompleto' 'Indefinido' nan
 'Doutorado Completo']

Valores unicos da coluna 'GENERO': 
 ['Masculino' 'Feminino' nan 'NI']


Com base no critério de impacto, e por padronização do domínio das variaveis, decidimos retirar as linhas nulas de DATA ADMISSAO e JORANADA MENSAL.

In [17]:
df = df.dropna(subset=['DATA ADMISSAO'])
df = df.dropna(subset=['JORANADA MENSAL'])

No caso de GENERO e GRAU DE INSTRUCAO temos definido uma categoria para valores indefinidos ou não informados. vamos fazer essas categorias absorverem os NaN sem problemas de interferir em nossa análise.

Sobre a coluna DATA DESLIGAMENTO, entendemos que os nulos se referem as servidores ativos que realmente não possuem data de desligamento, então criaremos de forma especial uma categoria chamada "NP" (Não se aplica).

Agora, vamos tratar preencher os valores nulos com os valores que definimos.

In [18]:
df['GENERO'] = df['GENERO'].fillna('NI')
df['GRAU DE INSTRUCAO'] = df['GRAU DE INSTRUCAO'].fillna('Indefinido')
df['DATA DESLIGAMENTO'] = df['DATA DESLIGAMENTO'].fillna('NP')

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1398296 entries, 0 to 1398331
Data columns (total 31 columns):
 #   Column                         Non-Null Count    Dtype  
---  ------                         --------------    -----  
 0   ANO                            1398296 non-null  int64  
 1   MES                            1398296 non-null  int64  
 2   CPF                            1398296 non-null  object 
 3   MATRICULA                      1398296 non-null  int64  
 4   NOME                           1398296 non-null  object 
 5   CATEGORIA                      1398296 non-null  object 
 6   CARGO                          1398296 non-null  object 
 7   FUNCAO                         1398296 non-null  object 
 8   VENCIMENTO CARGO               1398296 non-null  float64
 9   VENCIMENTO FUNCAO              1398296 non-null  float64
 10  REMUNERACAO                    1398296 non-null  float64
 11  VENCIMENTO FERIAS              1398296 non-null  float64
 12  VENCIMENTO NATAL   

Análise Final sobre Colunas:

Vamos remover colunas que passam informaçãoes muita especificas, mas desnecessarias para uma análise estatística quantitativa e generalizada que pretendemos fazer. 

Exemplos: Colunas de valor salarial que passam grande parte do dataset zeradas.


In [24]:
# Vencimentos, Remuneração, Proventos
print('quantidades de linhas em que REMUNERACAO é zero:', sum(df['REMUNERACAO'] == 0))
print('\nquantidades de linhas em que VENCIMENTO FUNCAO é zero:', sum(df['VENCIMENTO FUNCAO'] == 0))
print('\nquantidades de linhas em que VENCIMENTO CARGO é zero:', sum(df['VENCIMENTO CARGO'] == 0))
print('\nquantidades de linhas em que VENCIMENTO FERIAS é zero:', sum(df['VENCIMENTO FERIAS'] == 0))
print('\nquantidades de linhas em que VENCIMENTO NATAL é zero:', sum(df['VENCIMENTO NATAL'] == 0))
print('\nquantidades de linhas em que VENCIMENTOS OUTROS é zero:', sum(df['VENCIMENTOS OUTROS'] == 0))
print('\nquantidades de linhas em que DIFERENCA VENCIMENTO MES é zero:', sum(df['DIFERENCA VENCIMENTO MES'] == 0))
print('\nquantidades de linhas em que VENCIMENTO LIQUIDO é zero:', sum(df['VENCIMENTO LIQUIDO'] == 0))

# Descontos
print('\nquantidades de linhas em que DESCONTOS EXCEDENTES é zero:', sum(df['DESCONTOS EXCEDENTES'] == 0))
print('\nquantidades de linhas em que DESCONTO FALTA OU RESTITUICAO é zero:', sum(df['DESCONTO FALTA OU RESTITUICAO'] == 0))
print('\nquantidades de linhas em que DESCONTO PREVIDENCIA é zero:', sum(df['DESCONTO PREVIDENCIA'] == 0))
print('\nquantidades de linhas em que DESCONTO IRRF é zero:', sum(df['DESCONTO IRRF'] == 0))
print('\nquantidades de linhas em que DESCONTOS COMPULSORIOS é zero:', sum(df['DESCONTOS COMPULSORIOS'] == 0))

quantidades de linhas em que REMUNERACAO é zero: 292118

quantidades de linhas em que VENCIMENTO FUNCAO é zero: 1326267

quantidades de linhas em que VENCIMENTO CARGO é zero: 354832

quantidades de linhas em que VENCIMENTO FERIAS é zero: 1326372

quantidades de linhas em que VENCIMENTO NATAL é zero: 1299059

quantidades de linhas em que VENCIMENTOS OUTROS é zero: 121471

quantidades de linhas em que DIFERENCA VENCIMENTO MES é zero: 1224113

quantidades de linhas em que VENCIMENTO LIQUIDO é zero: 6746

quantidades de linhas em que DESCONTOS EXCEDENTES é zero: 1392451

quantidades de linhas em que DESCONTO FALTA OU RESTITUICAO é zero: 1370779

quantidades de linhas em que DESCONTO PREVIDENCIA é zero: 338260

quantidades de linhas em que DESCONTO IRRF é zero: 571077

quantidades de linhas em que DESCONTOS COMPULSORIOS é zero: 267561


percebemos que algumas colunas passam em torno de 90% do total de linhas, zeradas. Optamos por remover essas colunas para deixar nosso dataset mais limpo e objetivo.



In [25]:
checkpoint_2 = df.copy()

In [26]:
col_drop = ['VENCIMENTO FUNCAO', 'VENCIMENTO FERIAS', 'VENCIMENTO NATAL', 'DIFERENCA VENCIMENTO MES', 'DESCONTOS EXCEDENTES', 'DESCONTO FALTA OU RESTITUICAO']

df = df.drop(columns=col_drop, axis=1) #Removendo as colunas

RESULTADO FINAL

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1398296 entries, 0 to 1398331
Data columns (total 25 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   ANO                     1398296 non-null  int64  
 1   MES                     1398296 non-null  int64  
 2   CPF                     1398296 non-null  object 
 3   MATRICULA               1398296 non-null  int64  
 4   NOME                    1398296 non-null  object 
 5   CATEGORIA               1398296 non-null  object 
 6   CARGO                   1398296 non-null  object 
 7   FUNCAO                  1398296 non-null  object 
 8   VENCIMENTO CARGO        1398296 non-null  float64
 9   REMUNERACAO             1398296 non-null  float64
 10  VENCIMENTOS OUTROS      1398296 non-null  float64
 11  TOTAL PROVENTOS         1398296 non-null  float64
 12  DESCONTO PREVIDENCIA    1398296 non-null  float64
 13  DESCONTO IRRF           1398296 non-null  float64
 14  DESCONT